In [1]:
import os
import json
from typing import Final
from yfpy.query import YahooFantasySportsQuery
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import uuid

auth_dir = "/Users/petepritchard/Documents/Projects/ff-league-app/bot/auth"

# Load environment variables from .env file
load_dotenv()

# Fetch Yahoo API credentials from environment variables
CONSUMER_KEY: Final[str] = os.getenv('CONSUMER_KEY')
CONSUMER_SECRET: Final[str] = os.getenv('CONSUMER_SECRET')
REFRESH_TOKEN: Final[str] = os.getenv('REFRESH_TOKEN')

# Fetch league information
query = YahooFantasySportsQuery(
    auth_dir, 
    league_id='49754', # Fill in league_id
    game_code='nfl',   # Leave this as is
    game_id='423',     # Fill in current game id
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    )

/Users/petepritchard/Documents/Projects/ff-league-app/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [27]:
transactions = query.get_league_transactions()
recent_transaction = transactions[1].to_json()
data = json.loads(recent_transaction)

# Retrieving the values
player_id = data['players']['player']['player_id']
destination_team_key = data['players']['player']['transaction_data']['destination_team_key']
transaction_type = data['players']['player']['transaction_data']['type']

# Printing the retrieved values
print("Player ID:", player_id)
print("Destination Team Key:", destination_team_key)
print("Transaction Type:", transaction_type)

## Pulling data from Yahoo for Matchups Table

In [135]:
# Initialize an empty list to store records
records = []

for i in range(1, 7):
    # Query data from yahoo
    matchups = query.get_league_matchups_by_week(i)
    # Loop through matchup data
    for i in range(len(matchups)):
        matchup = matchups[i].to_json() # matchup[i] is a yfpy object
        matchup_dict = json.loads(matchup)
        
        # Generate a unique matchup_id (UUID)
        matchup_id = str(uuid.uuid4())

        # Extract information for each team in the matchup
        for team in matchup_dict.get('teams', []):
            team_info = team.get('team', {})
            record = {
                'matchups_key': str(uuid.uuid4()),  # Unique primary key for each row
                'matchup_id': matchup_id,
                'season_key': team_info.get('team_key')[:3], # First 3 characters in team key
                'week': matchup_dict.get('week'),
                'is_consolation': matchup_dict.get('is_consolation'),
                'is_playoffs': matchup_dict.get('is_playoffs'),
                'is_tied': matchup_dict.get('is_tied'),
                'winner_team_key': matchup_dict.get('winner_team_key'),
                'team_key': team_info.get('team_key'),
                'team_total_points': team_info.get('team_points', {}).get('total'),
                'team_projected_points': team_info.get('team_projected_points', {}).get('total')
            }
            
            # Add the record to the list
            records.append(record)

# Convert the list of records to a DataFrame
df = pd.DataFrame(records)

# Display the DataFrame
print(df.head())

                           matchups_key                            matchup_id  \
0  aac8e925-0758-4b0a-ab47-2e58f7f0f44a  40a7e589-7cdd-4f61-8ca5-af3f052f22a7   
1  6013dff4-57c0-4acf-9566-7fc248f5b732  40a7e589-7cdd-4f61-8ca5-af3f052f22a7   
2  ddd42cf8-e213-4428-8293-9ac872206389  d02ddf61-73d7-4e42-b813-1e7a790b50b5   
3  355d26e6-345c-4a3f-8613-d467c0c3a8a2  d02ddf61-73d7-4e42-b813-1e7a790b50b5   
4  6b729133-d805-4856-a3b7-c672dd1a55aa  32a9490f-ca9a-4bf6-aa3f-6edb0c18abb4   

  season_key  week  is_consolation  is_playoffs  is_tied  winner_team_key  \
0        423     1               0            0        0  423.l.49754.t.7   
1        423     1               0            0        0  423.l.49754.t.7   
2        423     1               0            0        0  423.l.49754.t.2   
3        423     1               0            0        0  423.l.49754.t.2   
4        423     1               0            0        0  423.l.49754.t.3   

          team_key  team_total_points  team_projec

In [136]:
# Creating win column 
df["win"] = np.where(df['winner_team_key'] == df['team_key'], 1, 0)
df.head()

,matchups_key,matchup_id,season_key,week,is_consolation,is_playoffs,is_tied,winner_team_key,team_key,team_total_points,team_projected_points,win
0,aac8e925-0758-4b0a-ab47-2e58f7f0f44a,40a7e589-7cdd-4f61-8ca5-af3f052f22a7,423,1,0,0,0,423.l.49754.t.7,423.l.49754.t.1,85.92,111.36,0
1,6013dff4-57c0-4acf-9566-7fc248f5b732,40a7e589-7cdd-4f61-8ca5-af3f052f22a7,423,1,0,0,0,423.l.49754.t.7,423.l.49754.t.7,97.00,107.88,1
2,ddd42cf8-e213-4428-8293-9ac872206389,d02ddf61-73d7-4e42-b813-1e7a790b50b5,423,1,0,0,0,423.l.49754.t.2,423.l.49754.t.2,121.68,116.17,1
3,355d26e6-345c-4a3f-8613-d467c0c3a8a2,d02ddf61-73d7-4e42-b813-1e7a790b50b5,423,1,0,0,0,423.l.49754.t.2,423.l.49754.t.8,102.37,117.43,0
4,6b729133-d805-4856-a3b7-c672dd1a55aa,32a9490f-ca9a-4bf6-aa3f-6edb0c18abb4,423,1,0,0,0,423.l.49754.t.3,423.l.49754.t.3,129.64,106.63,1


In [137]:
# Calculate average points and standard deviation for each team
team_stats = df.groupby('team_key').agg({
    'team_total_points': ['mean', 'std'],
    'team_projected_points': 'mean',
    'win': 'sum' 
}).reset_index()


team_stats.columns = ['team_key', 'avg_points', 'std_points', 'avg_projected_points', 'total_wins']

print(team_stats)

            team_key  avg_points  std_points  avg_projected_points  total_wins
0    423.l.49754.t.1  115.823333   23.411459            114.268333           3
1   423.l.49754.t.10  111.165000   27.776000            110.608333           3
2   423.l.49754.t.11  114.721667   19.000789            112.485000           5
3   423.l.49754.t.12  122.055000   44.387356            117.776667           4
4    423.l.49754.t.2  111.735000   17.589390            109.476667           2
5    423.l.49754.t.3  105.088333   18.719453            106.998333           1
6    423.l.49754.t.4  109.930000   13.372692            109.406667           2
7    423.l.49754.t.5  106.951667   35.308796            115.390000           2
8    423.l.49754.t.6  120.093333   49.135549            113.321667           2
9    423.l.49754.t.7  118.945000   20.124323            115.911667           5
10   423.l.49754.t.8  107.231667   17.886818            111.978333           3
11   423.l.49754.t.9  121.888333   21.330973        

In [ ]:
# Parameters
n_simulations = 10000  # Number of Monte Carlo simulations
n_weeks_remaining = 8  # Number of weeks remaining in the season
n_playoff_teams = 8    # Number of teams that make the playoffs

In [ ]:
matchups_table = """
                CREATE TABLE Matchups (
                    matchups_key BIGINT PRIMARY KEY,
                    matchup_id BIGINT,
                    season_key INT NOT NULL,
                    week INT NOT NULL,
                    is_consolation BOOLEAN,
                    is_playoffs BOOLEAN,
                    is_tied BOOLEAN,
                    winner_team_key VARCHAR(50),
                    team_key VARCHAR(50),
                    team_total_points FLOAT,
                    team_projected_points FLOAT,
                    UNIQUE(season, week, team_key) -- Ensure a team has one matchup per week
                    FOREIGN KEY (season_key) REFERENCES Seasons(season_key), -- Seasons table
                    FOREIGN KEY (team_key) REFERENCES Teams(team_key) -- Teams table
                );
                """

## Seasons Table

In [94]:
seasons_dict = {
    "season_key": ['257', '273', '314', '331', '348', '259', '317', '380', '380', '399', '406', '414', '423', '449'],
    "season": ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'],
}

df_seasons = pd.DataFrame(seasons_dict)
df_seasons.head()

,season_key,season
0,257,2011
1,273,2012
2,314,2013
3,331,2014
4,348,2015


In [ ]:
seasons_table = """
                CREATE TABLE Seasons (
                    season_key INT PRIMARY KEY,
                    season_year INT NOT NULL
                    UNIQUE(season) -- Ensuring each year is unique
                );
                """

## Drafts Table

In [99]:
draft = query.get_league_draft_results()
print(type(draft[1]))

<class 'yfpy.models.DraftResult'>


In [100]:
print(draft[1])

DraftResult({
  "pick": 2,
  "round": 1,
  "team_key": "423.l.49754.t.6",
  "player_key": "423.p.33393"
})


In [104]:
records = []

for i in range(len(draft)):
    pick = draft[i].to_json()
    pick_dict = json.loads(pick)
    record = {
        'draft_id': str(uuid.uuid4()),
        'pick_number': pick_dict.get('pick'),
        'round': pick_dict.get('round'),
        'team_key': pick_dict.get('team_key'),
        'player_key': pick_dict.get('player_key')
    }

    records.append(record)

# Convert the list of records to a DataFrame
df = pd.DataFrame(records)

# Display the DataFrame
print(df.head())    

                               draft_id  pick  round          team_key  \
0  14c08d76-5222-4b87-9791-3bcf4247ae75     1      1  423.l.49754.t.12   
1  aa9b6347-48b4-4a8f-950f-3522b81908ab     2      1   423.l.49754.t.6   
2  281d24bc-bb6c-43f0-896b-5df49067222b     3      1   423.l.49754.t.4   
3  0f0ab24c-5ccf-438b-85bb-9f28638adea7     4      1  423.l.49754.t.10   
4  6ea91085-0eb2-44cc-a9cc-b5dc1d88a9dc     5      1   423.l.49754.t.2   

    player_key  
0  423.p.30121  
1  423.p.33393  
2  423.p.32692  
3  423.p.31005  
4  423.p.30423  


In [ ]:
drafts_table = """
                CREATE TABLE Drafts (
                    draft_id VARCHAR(50) PRIMARY KEY,
                    pick_number INT,
                    round INT,
                    team_key VARCHAR(50),
                    player_key VARCHAR(50),
                    UNIQUE (round, pick_number, team_key), -- Ensuring no duplicate picks for the same team in a round
                    INDEX (team_key), -- Adding index for faster lookups
                    INDEX (player_key) -- Adding index for faster lookups
                );
                """

## Players Table

In [ ]:
players = query.get_league_players()

In [113]:
records = []

for i in range(len(players)):
    player = players[i].to_json()
    player_dict = json.loads(player)
    record = {
        'player_id': str(uuid.uuid4()),
        'player_key': player_dict.get('player_key'),
        'season_key': int(player_dict.get('player_key')[:3]),
        'full_name': player_dict.get('name', {}).get('full'),
        'bye_week': player_dict.get('bye_weeks', {}).get('week'),
        'position': player_dict.get('display_position'),
        'team_abbr': player_dict.get('editorial_team_abbr'),
        'status': player_dict.get('status')
    }

    records.append(record)
    
# Convert the list of records to a DataFrame
df = pd.DataFrame(records)

# Display the DataFrame
print(df.head())   

   player_key       full_name  bye_week position team_abbr status
0  423.p.5228       Tom Brady         5       QB        TB     NA
1  423.p.7200   Aaron Rodgers         7       QB       NYJ      Q
2  423.p.7520    Robbie Gould         9        K        SF     NA
3  423.p.7777  Marcedes Lewis        13       TE       Chi   None
4  423.p.8432       Nick Folk         7        K       Ten   None


In [114]:
players_table = """
                CREATE TABLE (
                    player_id VARCHAR(50) PRIMARY KEY,
                    player_key VARCHAR(50),
                    season_key INT NOT NULL,
                    full_name VARCHAR(50) NOT NULL,
                    bye_week INT,
                    position VARCHAR(50) NOT NULL,
                    team_abbr VARCHAR(50) NOT NULL,
                    status VARCHAR(50)
                    INDEX (position),  -- Index for faster queries by position
                    INDEX (team_abbr)  -- Index for faster queries by team abbreviation
                );
                """

## Managers Table

In [120]:
managers = query.get_league_teams()
print(managers[1])

Team({
  "clinched_playoffs": 1,
  "draft_grade": "D+",
  "draft_recap_url": "https://football.fantasysports.yahoo.com/2023/f1/49754/2/draftrecap",
  "faab_balance": 0,
  "has_draft_grade": 1,
  "league_scoring_type": "head",
  "managers": {
    "manager": {
      "felo_score": 687,
      "felo_tier": "silver",
      "guid": "7AK7KQ54ASSIFTBFGM46KSYIJH",
      "image_url": "https://s.yimg.com/ag/images/default_user_profile_pic_64sq.jpg",
      "manager_id": 2,
      "nickname": "lewis"
    }
  },
  "name": "De most Ernest Lewis",
  "number_of_moves": 28,
  "number_of_trades": 5,
  "roster_adds": {
    "coverage_type": "week",
    "coverage_value": 18,
    "value": 0
  },
  "team_id": 2,
  "team_key": "423.l.49754.t.2",
  "team_logos": {
    "team_logo": {
      "size": "large",
      "url": "https://yahoofantasysports-res.cloudinary.com/image/upload/t_s192sq/fantasy-logos/39202166743_26a06a.jpg"
    }
  },
  "url": "https://football.fantasysports.yahoo.com/2023/f1/49754/2",
  "waiver_p

In [126]:
records = []

for i in range(len(managers)):
    manager = managers[i].to_json()
    manager_dict = json.loads(manager)
    record = {
        'manager_id': str(uuid.uuid4()),
        'team_key': manager_dict.get('team_key'),
        'season_key': int(manager_dict.get('team_key')[:3]),
        'team_name': manager_dict.get('name'),
        'nickname': manager_dict.get("managers", {}).get("manager", {}).get("nickname"),
        'felo_score': manager_dict.get("managers", {}).get("manager", {}).get("felo_score"),
        'felo_tier': manager_dict.get("managers", {}).get("manager", {}).get("felo_tier"),
        'image_url': manager_dict.get("team_logos", {}).get("team_logo", {}).get("url"),
        'faab_balance': manager_dict.get('faab_balance'),
        'number_of_moves': manager_dict.get('number_of_moves'),
        'number_of_trades': manager_dict.get('number_of_trades'),
    }

    records.append(record)
    
# Convert the list of records to a DataFrame
df = pd.DataFrame(records)

# Display the DataFrame
print(df.head())   

                             manager_id         team_key  season_key  \
0  0ffebf3c-76ad-46de-837f-7a3085f9af58  423.l.49754.t.1         423   
1  e9892dac-7cec-4a6d-aca7-233c56acf2c1  423.l.49754.t.2         423   
2  c85e519e-a878-4d71-9318-1091153be8f1  423.l.49754.t.3         423   
3  e7df726d-b6c9-4d7e-ae6f-ea5a695bf1a1  423.l.49754.t.4         423   
4  2eec70c7-381d-4d97-8ab2-b732f20c2af9  423.l.49754.t.5         423   

              team_name  nickname  felo_score felo_tier  \
0      LeJizzleby James  Richmond         610    silver   
1  De most Ernest Lewis     lewis         687    silver   
2         aPuka maChuba   William         570    bronze   
3       Cruspy McNugent   Juzzman         690    silver   
4      Fupa McChoadslap  Marshall         530    bronze   

                                           image_url  faab_balance  \
0  https://yahoofantasysports-res.cloudinary.com/...             0   
1  https://yahoofantasysports-res.cloudinary.com/...             0   
2 

In [125]:
managers_table = """
                CREATE TABLE Managers (
                    manager_id VARCHER(50) PRIMARY KEY,
                    team_key VARCHAR(50) NOT NULL,
                    seaon_key INT NOT NULL,
                    team_name VARCHAR(50),
                    nickname VARCHAR(50),
                    felo_score INT,
                    felo_tier VARCHAR(50),
                    image_url VARCHAR(50),
                    faab_balance INT
                    UNIQUE (team_key, season_key),  -- Ensuring that a team is uniquely tied to a season
                    FOREIGN KEY (season_key) REFERENCES Seasons(season_key),
                    FOREIGN KEY (team_key) REFERENCES Teams(team_key)
                );
                """

In [3]:
from yahoo.functionality import get_current_week
from utils.scripts import map_team_key_to_nickname
# Get current week
current_week = get_current_week()

# Initialize an empty list to store records
records = []

for i in range(1, current_week + 1): 
    # Query data from yahoo
    matchups = query.get_league_matchups_by_week(i)
    # Loop through matchup data
    for i in range(len(matchups)):
        matchup = matchups[i].to_json() # matchup[i] is a yfpy object
        matchup_dict = json.loads(matchup)
        
        # Generate a unique matchup_id (UUID)
        matchup_id = str(uuid.uuid4())

        # Extract information for each team in the matchup
        for team in matchup_dict.get('teams', []):
            team_info = team.get('team', {})
            record = {
                'matchup_id': matchup_id,
                'week': matchup_dict.get('week'),
                'winner_team_key': matchup_dict.get('winner_team_key'),
                'team_key': team_info.get('team_key'),
                'team_total_points': team_info.get('team_points', {}).get('total'),
                'team_projected_points': team_info.get('team_projected_points', {}).get('total')
            }
            
            # Add the record to the list
            records.append(record)

# Convert the list of records to a DataFrame
df = pd.DataFrame(records)
# Creating win column 
df["win"] = np.where(df['winner_team_key'] == df['team_key'], 1, 0)

# Calculate average points and standard deviation for each team
team_stats = df.groupby('team_key').agg({
    'team_total_points': ['mean', 'std', 'sum', 'min', 'max'],
    'win': 'sum' 
}).reset_index()

# Set column names
team_stats.columns = ['team_key', 'avg_points', 'std_points', 'total_points', 'min_points', 'max_points', 'wins']
# Map team key to nickname
team_stats = map_team_key_to_nickname(team_stats, 'team_key')
# Calculate ranking
team_stats['p_rank'] = ((team_stats.avg_points * 6) + ((team_stats.max_points + team_stats.min_points) * 2) + (((team_stats.wins / current_week) * 200) * 2)) / 10
# grab only rank and nickname
p_rank_table = pd.DataFrame()
p_rank_table['Manager'] = team_stats['nickname']
p_rank_table['Power Score'] = team_stats['p_rank']
# Add a new 'Rank' column based on the Power Score
p_rank_table['Rank'] = p_rank_table['Power Score'].rank(ascending=False, method='min').astype(int)
# Sort by Rank if needed
p_rank_table = p_rank_table.sort_values('Rank')


,Manager,Power Score
6,Juzzman,150.189529
2,Hunter,148.760471
3,Parker,147.572353
11,tcudd,145.302588
8,Reed,140.760790
4,lewis,138.216471
1,Pete,134.116235
0,Richmond,132.598941
9,Patrick,126.871294
10,Ty Mcmahon,125.862790
